In [1]:
import sys
import os
import csv
import random
from keras.layers import *
from keras.optimizers import *
from keras.applications import *
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as k
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from force_utils import DataProcessing

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
csv_file = 'dataset/forceinfo.csv'
df = pd.read_csv(csv_file)
truncation = -7
df_time = df['Time'].as_matrix()[:truncation]
df_target = df.drop(["Time"], axis=1).as_matrix()[:truncation]

In [4]:
target = df_target.copy()

In [17]:
train_num = int(round(0.8 * len(target)))
y_train = target[:train_num]
y_test = target[train_num:]

In [18]:
print(y_train.shape, y_test.shape, y_train.shape[0]+y_test.shape[0])

(45289, 6) (11322, 6) 56611


In [11]:
x_train = np.load('./dataset/features/vgg16_train.npy')
x_test = np.load('./dataset/features/vgg16_val.npy')

In [12]:
print(x_train.shape, x_test.shape, x_train.shape[0]+x_test.shape[0])

(45312, 9, 9, 512) (11328, 9, 9, 512) 56640


In [19]:
x_train = x_train[:-23]
x_test = x_test[:-6]
print(x_train.shape, x_test.shape, x_train.shape[0]+x_test.shape[0])

(45289, 9, 9, 512) (11322, 9, 9, 512) 56611


In [20]:
# -------------------------------------------------------------------------------------
# VARIABLE
# -------------------------------------------------------------------------------------

input_shape = (299, 299, 3)
batch_size = 32
epochs = 100
learning_rate = 0.01

In [21]:
# -------------------------------------------------------------------------------------
# MODEL
# -------------------------------------------------------------------------------------

input_tensor = Input(shape=x_train.shape[1:])
x = Flatten(input_shape=x_train.shape[1:])(input_tensor)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(6)(x)
model = Model(inputs=input_tensor, outputs=prediction)

In [22]:
# -------------------------------------------------------------------------------------
# COMPILE
# -------------------------------------------------------------------------------------

# Optimizer
optim = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

# Learning rate scheduler
def schedule(epoch, decay=0.9):
    return learning_rate * decay**(epoch)

# Callbacks
callbacks = [
    ModelCheckpoint('dataset/weights/six_{epoch:02d}.h5', monitor='val_loss', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')]
    # LearningRateScheduler(schedule)]

# Compile
model.compile(optimizer=optim, loss='mean_squared_error')

In [23]:
# -------------------------------------------------------------------------------------
# FIT
# -------------------------------------------------------------------------------------

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=(x_test, y_test)
)

Train on 45289 samples, validate on 11322 samples
Epoch 1/100
45289/45289 [==============================] - 24s 527us/step - loss: 1.8340 - val_loss: 0.1338

Epoch 00001: val_loss improved from inf to 0.13382, saving model to dataset/weights/six_01.h5
Epoch 2/100
45289/45289 [==============================] - 23s 507us/step - loss: 0.1668 - val_loss: 0.0970

Epoch 00002: val_loss improved from 0.13382 to 0.09702, saving model to dataset/weights/six_02.h5
Epoch 3/100
45289/45289 [==============================] - 23s 507us/step - loss: 0.1374 - val_loss: 0.1215

Epoch 00003: val_loss did not improve from 0.09702
Epoch 4/100
45289/45289 [==============================] - 23s 509us/step - loss: 0.1219 - val_loss: 0.0943

Epoch 00004: val_loss improved from 0.09702 to 0.09429, saving model to dataset/weights/six_04.h5
Epoch 5/100
45289/45289 [==============================] - 23s 508us/step - loss: 0.1071 - val_loss: 0.1195

Epoch 00005: val_loss did not improve from 0.09429
Epoch 6/100
4

NameError: name 'save_history' is not defined